In [1]:
# Import the libraries.

# Import pandas.
import pandas as pd
# Import numpy.
import numpy as np
# Import matplotlib.
import matplotlib.pyplot as plt
# Import seaborn.
import seaborn as sns
# Import regular expression.
import re
# import datetime.
import datetime as dt
# Import the data.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [3]:
df = pd.read_csv(r'C:\Users\USER\phase2project2\pacific.csv')
df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,EP011949,UNNAMED,19490611,0,,TS,20.2N,106.3W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,EP011949,UNNAMED,19490611,600,,TS,20.2N,106.4W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,EP011949,UNNAMED,19490611,1200,,TS,20.2N,106.7W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,EP011949,UNNAMED,19490611,1800,,TS,20.3N,107.7W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,EP011949,UNNAMED,19490612,0,,TS,20.4N,108.6W,45,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,,LO,21.7N,109.0W,35,1002,...,0,0,0,0,0,0,0,0,0,0
26133,EP222015,SANDRA,20151128,1800,,LO,22.4N,108.7W,30,1007,...,0,0,0,0,0,0,0,0,0,0
26134,EP222015,SANDRA,20151129,0,,LO,23.1N,108.3W,30,1008,...,0,0,0,0,0,0,0,0,0,0
26135,EP222015,SANDRA,20151129,600,,LO,23.5N,107.9W,25,1009,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#removing Hemisphere tags from coordiantes:
df['Longitude'] = df['Longitude'].map(lambda x: x.rstrip('W'))
df['Latitude'] = df['Latitude'].map(lambda x: x.rstrip('N'))
df['Latitude'] = df['Latitude'].map(lambda x: x.rstrip('S'))
df['Longitude'] = df['Longitude'].map(lambda x: x.rstrip('E'))
#converting Latitude and Longitude to floats:
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [5]:
# Show the count of missing values and fill them with mean.
for column in df.columns:
    missing_cnt = df[column][df[column] == -999].count()
    print('Missing Values in column {col} = '.format(col = column) , missing_cnt )
    if missing_cnt!= 0:
#         print('in ' , column)
        mean = round(df[column][df[column] != -999 ].mean())
#         print("mean",mean)
        index = df.loc[df[column] == -999 , column].index
#         print("index" , index )
        df.loc[df[column] == -999 , column] = mean
#         print(df.loc[index , column])

Missing Values in column ID =  0
Missing Values in column Name =  0
Missing Values in column Date =  0
Missing Values in column Time =  0
Missing Values in column Event =  0
Missing Values in column Status =  0
Missing Values in column Latitude =  0
Missing Values in column Longitude =  0
Missing Values in column Maximum Wind =  0
Missing Values in column Minimum Pressure =  12804
Missing Values in column Low Wind NE =  19750
Missing Values in column Low Wind SE =  19750
Missing Values in column Low Wind SW =  19750
Missing Values in column Low Wind NW =  19750
Missing Values in column Moderate Wind NE =  19750
Missing Values in column Moderate Wind SE =  19750
Missing Values in column Moderate Wind SW =  19750
Missing Values in column Moderate Wind NW =  19750
Missing Values in column High Wind NE =  19750
Missing Values in column High Wind SE =  19750
Missing Values in column High Wind SW =  19750
Missing Values in column High Wind NW =  19750


In [6]:
# Restructure the dataframe for visibility and remove columns ID and Event.        
df =  df[['ID', 'Name', 'Date', 'Time', 'Event', 'Status', 'Latitude',  
       'Longitude' ,'Maximum Wind', 'Minimum Pressure', 'Low Wind NE',
       'Low Wind SE', 'Low Wind SW', 'Low Wind NW', 'Moderate Wind NE',
       'Moderate Wind SE', 'Moderate Wind SW', 'Moderate Wind NW',
       'High Wind NE', 'High Wind SE', 'High Wind SW', 'High Wind NW']]

In [7]:
# Change all time to format HHMM.
df['Time'] = df['Time'].astype('object')
def hhmm(time):
    time = str(time)
    digits = re.findall(r'\d', time)
    t = ''
    if len(digits) == 1:
        t ='0{i}00'.format(i =time)
    elif len(digits) == 2:
        t = '{i}00'.format(i =time)
    elif len(digits) == 3:
        t = '0{i}'.format(i =time)
    else:
        t = time
    return t

In [8]:
# Apply the function.
df['Time'] = df['Time'].apply(hhmm)



In [9]:
df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,EP011949,UNNAMED,19490611,0000,,TS,20.2,106.3,45,995,...,27,35,13,11,9,11,5,4,3,5
1,EP011949,UNNAMED,19490611,0600,,TS,20.2,106.4,45,995,...,27,35,13,11,9,11,5,4,3,5
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,106.7,45,995,...,27,35,13,11,9,11,5,4,3,5
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,107.7,45,995,...,27,35,13,11,9,11,5,4,3,5
4,EP011949,UNNAMED,19490612,0000,,TS,20.4,108.6,45,995,...,27,35,13,11,9,11,5,4,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,,LO,21.7,109.0,35,1002,...,0,0,0,0,0,0,0,0,0,0
26133,EP222015,SANDRA,20151128,1800,,LO,22.4,108.7,30,1007,...,0,0,0,0,0,0,0,0,0,0
26134,EP222015,SANDRA,20151129,0000,,LO,23.1,108.3,30,1008,...,0,0,0,0,0,0,0,0,0,0
26135,EP222015,SANDRA,20151129,0600,,LO,23.5,107.9,25,1009,...,0,0,0,0,0,0,0,0,0,0


In [10]:



# Convert the status column to categorical.
df['Status'] = df['Status'].astype('category')


In [11]:
df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,EP011949,UNNAMED,19490611,0000,,TS,20.2,106.3,45,995,...,27,35,13,11,9,11,5,4,3,5
1,EP011949,UNNAMED,19490611,0600,,TS,20.2,106.4,45,995,...,27,35,13,11,9,11,5,4,3,5
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,106.7,45,995,...,27,35,13,11,9,11,5,4,3,5
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,107.7,45,995,...,27,35,13,11,9,11,5,4,3,5
4,EP011949,UNNAMED,19490612,0000,,TS,20.4,108.6,45,995,...,27,35,13,11,9,11,5,4,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,,LO,21.7,109.0,35,1002,...,0,0,0,0,0,0,0,0,0,0
26133,EP222015,SANDRA,20151128,1800,,LO,22.4,108.7,30,1007,...,0,0,0,0,0,0,0,0,0,0
26134,EP222015,SANDRA,20151129,0000,,LO,23.1,108.3,30,1008,...,0,0,0,0,0,0,0,0,0,0
26135,EP222015,SANDRA,20151129,0600,,LO,23.5,107.9,25,1009,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data = df.drop(columns = ['ID' , 'Event'])

In [13]:
# Import Decision Tree Classifier.
from sklearn.tree import DecisionTreeClassifier

# Import train-test split.
from sklearn.model_selection import train_test_split

# Import accuracy Score.
from sklearn.metrics import accuracy_score

#Import Recall Score.
from sklearn.metrics import recall_score 

#Import Precision Score.
from sklearn.metrics import precision_score 

# Form the model.
dt = DecisionTreeClassifier(min_samples_leaf=50 , criterion='entropy')

In [14]:
# Set the dependent and independent variables.
x_train = data[['Latitude', 
       'Longitude', 'Maximum Wind', 'Minimum Pressure',
       'Low Wind NE', 'Low Wind SE', 'Low Wind SW', 'Low Wind NW',
       'Moderate Wind NE', 'Moderate Wind SE', 'Moderate Wind SW',
       'Moderate Wind NW', 'High Wind NE', 'High Wind SE', 'High Wind SW',
       'High Wind NW' ]]
y_train = data['Status']

In [15]:
df['Wind NW']=(df['Low Wind NW']+df['Moderate Wind NW']+df['High Wind NW'])/3
df['Wind NE']=(df['Low Wind NE']+df['Moderate Wind NE']+df['High Wind NE'])/3

In [16]:
df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Wind NW,Wind NE
0,EP011949,UNNAMED,19490611,0000,,TS,20.2,106.3,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
1,EP011949,UNNAMED,19490611,0600,,TS,20.2,106.4,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,106.7,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,107.7,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
4,EP011949,UNNAMED,19490612,0000,,TS,20.4,108.6,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,,LO,21.7,109.0,35,1002,...,0,0,0,0,0,0,0,0,0.0,20.000000
26133,EP222015,SANDRA,20151128,1800,,LO,22.4,108.7,30,1007,...,0,0,0,0,0,0,0,0,0.0,0.000000
26134,EP222015,SANDRA,20151129,0000,,LO,23.1,108.3,30,1008,...,0,0,0,0,0,0,0,0,0.0,0.000000
26135,EP222015,SANDRA,20151129,0600,,LO,23.5,107.9,25,1009,...,0,0,0,0,0,0,0,0,0.0,0.000000


In [17]:
# Perform the Kfold validation.

# Import the KFold library.
from sklearn.model_selection import KFold
kf = KFold(n_splits=10 , shuffle= True , random_state=42 )

dt_scores = []
dt_recall_scores = []
dt_precision_scores = []
for tr , ts in kf.split(x_train):
    xtr = x_train.loc[tr]
    ytr = y_train.loc[tr]
    xts = x_train.loc[ts]
    yts = y_train.loc[ts]
    dt.fit(xtr , ytr)
    y_pred = dt.predict(xts) 
    dt_scores.append(accuracy_score(yts, y_pred)) 
    dt_recall_scores.append(recall_score(yts , y_pred , average = 'weighted'))
    dt_precision_scores.append(precision_score(yts , y_pred , average = 'weighted'))
# dt.fit(x_train, y_train)
# y_pred = dt.predict(x_test)
# accuracy_score(y_test, y_pred)
dt_scr = {'accuracy' : np.mean(dt_scores) , 'recall': np.mean(dt_recall_scores) , 'precision' :  np.mean(dt_precision_scores) }
print('Accuracy score for Decision Tree is :' , dt_scr['accuracy'])
print('Recall score for Decision Tree is :' , dt_scr['recall'])
print('Precision score for Decision Tree is :' , dt_scr['precision'])

H:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning

Accuracy score for Decision Tree is : 0.9613570954011064
Recall score for Decision Tree is : 0.9613570954011064
Precision score for Decision Tree is : 0.9589016920940393


H:\anaconda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
df

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Wind NW,Wind NE
0,EP011949,UNNAMED,19490611,0000,,TS,20.2,106.3,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
1,EP011949,UNNAMED,19490611,0600,,TS,20.2,106.4,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,106.7,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,107.7,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
4,EP011949,UNNAMED,19490612,0000,,TS,20.4,108.6,45,995,...,13,11,9,11,5,4,3,5,17.0,19.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,EP222015,SANDRA,20151128,1200,,LO,21.7,109.0,35,1002,...,0,0,0,0,0,0,0,0,0.0,20.000000
26133,EP222015,SANDRA,20151128,1800,,LO,22.4,108.7,30,1007,...,0,0,0,0,0,0,0,0,0.0,0.000000
26134,EP222015,SANDRA,20151129,0000,,LO,23.1,108.3,30,1008,...,0,0,0,0,0,0,0,0,0.0,0.000000
26135,EP222015,SANDRA,20151129,0600,,LO,23.5,107.9,25,1009,...,0,0,0,0,0,0,0,0,0.0,0.000000


In [19]:
#drop unnecessary titles
df=df.drop(['ID', 'Time' ,'Name', 'Event'], axis = 1)

In [20]:
df=df.drop(['Date'], axis = 1)

In [21]:
df=df.drop(['Maximum Wind', 'Low Wind NE' ,'Low Wind NE', 'Low Wind NE', 'Low Wind NE'], axis = 1)

In [22]:
df

,Status,Latitude,Longitude,Minimum Pressure,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Wind NW,Wind NE
0,TS,20.2,106.3,995,35,27,35,13,11,9,11,5,4,3,5,17.0,19.333333
1,TS,20.2,106.4,995,35,27,35,13,11,9,11,5,4,3,5,17.0,19.333333
2,TS,20.2,106.7,995,35,27,35,13,11,9,11,5,4,3,5,17.0,19.333333
3,TS,20.3,107.7,995,35,27,35,13,11,9,11,5,4,3,5,17.0,19.333333
4,TS,20.4,108.6,995,35,27,35,13,11,9,11,5,4,3,5,17.0,19.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,LO,21.7,109.0,1002,0,0,0,0,0,0,0,0,0,0,0,0.0,20.000000
26133,LO,22.4,108.7,1007,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000
26134,LO,23.1,108.3,1008,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000
26135,LO,23.5,107.9,1009,0,0,0,0,0,0,0,0,0,0,0,0.0,0.000000


In [23]:
df=df.drop(['High Wind NW', 'Low Wind NW' ,'Moderate Wind NE', 'Moderate Wind NW', 'High Wind NE','High Wind NW'], axis = 1)

In [24]:
df

,Status,Latitude,Longitude,Minimum Pressure,Low Wind SE,Low Wind SW,Moderate Wind SE,Moderate Wind SW,High Wind SE,High Wind SW,Wind NW,Wind NE
0,TS,20.2,106.3,995,35,27,11,9,4,3,17.0,19.333333
1,TS,20.2,106.4,995,35,27,11,9,4,3,17.0,19.333333
2,TS,20.2,106.7,995,35,27,11,9,4,3,17.0,19.333333
3,TS,20.3,107.7,995,35,27,11,9,4,3,17.0,19.333333
4,TS,20.4,108.6,995,35,27,11,9,4,3,17.0,19.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
26132,LO,21.7,109.0,1002,0,0,0,0,0,0,0.0,20.000000
26133,LO,22.4,108.7,1007,0,0,0,0,0,0,0.0,0.000000
26134,LO,23.1,108.3,1008,0,0,0,0,0,0,0.0,0.000000
26135,LO,23.5,107.9,1009,0,0,0,0,0,0,0.0,0.000000


In [25]:
df['Wind SW']=(df['Low Wind SW']+df['Moderate Wind SW']+df['High Wind SW'])/3
df['Wind SE']=(df['Low Wind SE']+df['Moderate Wind SE']+df['High Wind SE'])/3

In [26]:
df

,Status,Latitude,Longitude,Minimum Pressure,Low Wind SE,Low Wind SW,Moderate Wind SE,Moderate Wind SW,High Wind SE,High Wind SW,Wind NW,Wind NE,Wind SW,Wind SE
0,TS,20.2,106.3,995,35,27,11,9,4,3,17.0,19.333333,13.0,16.666667
1,TS,20.2,106.4,995,35,27,11,9,4,3,17.0,19.333333,13.0,16.666667
2,TS,20.2,106.7,995,35,27,11,9,4,3,17.0,19.333333,13.0,16.666667
3,TS,20.3,107.7,995,35,27,11,9,4,3,17.0,19.333333,13.0,16.666667
4,TS,20.4,108.6,995,35,27,11,9,4,3,17.0,19.333333,13.0,16.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26132,LO,21.7,109.0,1002,0,0,0,0,0,0,0.0,20.000000,0.0,0.000000
26133,LO,22.4,108.7,1007,0,0,0,0,0,0,0.0,0.000000,0.0,0.000000
26134,LO,23.1,108.3,1008,0,0,0,0,0,0,0.0,0.000000,0.0,0.000000
26135,LO,23.5,107.9,1009,0,0,0,0,0,0,0.0,0.000000,0.0,0.000000


In [27]:
df=df.drop(['High Wind SW', 'Low Wind SW' ,'Moderate Wind SE', 'Moderate Wind SW', 'High Wind SE','High Wind SW'], axis = 1)

In [28]:
df

,Status,Latitude,Longitude,Minimum Pressure,Low Wind SE,Wind NW,Wind NE,Wind SW,Wind SE
0,TS,20.2,106.3,995,35,17.0,19.333333,13.0,16.666667
1,TS,20.2,106.4,995,35,17.0,19.333333,13.0,16.666667
2,TS,20.2,106.7,995,35,17.0,19.333333,13.0,16.666667
3,TS,20.3,107.7,995,35,17.0,19.333333,13.0,16.666667
4,TS,20.4,108.6,995,35,17.0,19.333333,13.0,16.666667
...,...,...,...,...,...,...,...,...,...
26132,LO,21.7,109.0,1002,0,0.0,20.000000,0.0,0.000000
26133,LO,22.4,108.7,1007,0,0.0,0.000000,0.0,0.000000
26134,LO,23.1,108.3,1008,0,0.0,0.000000,0.0,0.000000
26135,LO,23.5,107.9,1009,0,0.0,0.000000,0.0,0.000000


In [29]:
df=df.drop(['Low Wind SE' ], axis = 1)

In [30]:
df

,Status,Latitude,Longitude,Minimum Pressure,Wind NW,Wind NE,Wind SW,Wind SE
0,TS,20.2,106.3,995,17.0,19.333333,13.0,16.666667
1,TS,20.2,106.4,995,17.0,19.333333,13.0,16.666667
2,TS,20.2,106.7,995,17.0,19.333333,13.0,16.666667
3,TS,20.3,107.7,995,17.0,19.333333,13.0,16.666667
4,TS,20.4,108.6,995,17.0,19.333333,13.0,16.666667
...,...,...,...,...,...,...,...,...
26132,LO,21.7,109.0,1002,0.0,20.000000,0.0,0.000000
26133,LO,22.4,108.7,1007,0.0,0.000000,0.0,0.000000
26134,LO,23.1,108.3,1008,0.0,0.000000,0.0,0.000000
26135,LO,23.5,107.9,1009,0.0,0.000000,0.0,0.000000


In [31]:
df

,Status,Latitude,Longitude,Minimum Pressure,Wind NW,Wind NE,Wind SW,Wind SE
0,TS,20.2,106.3,995,17.0,19.333333,13.0,16.666667
1,TS,20.2,106.4,995,17.0,19.333333,13.0,16.666667
2,TS,20.2,106.7,995,17.0,19.333333,13.0,16.666667
3,TS,20.3,107.7,995,17.0,19.333333,13.0,16.666667
4,TS,20.4,108.6,995,17.0,19.333333,13.0,16.666667
...,...,...,...,...,...,...,...,...
26132,LO,21.7,109.0,1002,0.0,20.000000,0.0,0.000000
26133,LO,22.4,108.7,1007,0.0,0.000000,0.0,0.000000
26134,LO,23.1,108.3,1008,0.0,0.000000,0.0,0.000000
26135,LO,23.5,107.9,1009,0.0,0.000000,0.0,0.000000


In [32]:
df['Status'].value_counts()

 TS     10076
 TD      6965
 HU      6765
 LO      1828
 DB       217
 ET       152
 EX       110
 SS         7
 ST         6
 PT         6
 SD         4
 HU         1
Name: Status, dtype: int64

In [33]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Status']=le.fit_transform(df['Status'])

In [34]:
df['Status'].value_counts()

11    10076
10     6965
3      6765
5      1828
0       217
1       152
2       110
8         7
9         6
6         6
7         4
4         1
Name: Status, dtype: int64

In [35]:

y=df.iloc[:,0:1].values
x=df.iloc[:,1:8].values

In [36]:
x

array([[  20.2       ,  106.3       ,  995.        , ...,   19.33333333,
          13.        ,   16.66666667],
       [  20.2       ,  106.4       ,  995.        , ...,   19.33333333,
          13.        ,   16.66666667],
       [  20.2       ,  106.7       ,  995.        , ...,   19.33333333,
          13.        ,   16.66666667],
       ...,
       [  23.1       ,  108.3       , 1008.        , ...,    0.        ,
           0.        ,    0.        ],
       [  23.5       ,  107.9       , 1009.        , ...,    0.        ,
           0.        ,    0.        ],
       [  24.2       ,  107.7       , 1010.        , ...,    0.        ,
           0.        ,    0.        ]])

In [37]:
y

array([[11],
       [11],
       [11],
       ...,
       [ 5],
       [ 5],
       [ 5]])

In [38]:
from sklearn import preprocessing

In [39]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler(with_mean=False)
xl=sc.fit_transform(x)
xl

array([[ 4.37794766,  5.69605368, 76.86280565, ...,  1.49810239,
         1.36333434,  1.44377591],
       [ 4.37794766,  5.70141215, 76.86280565, ...,  1.49810239,
         1.36333434,  1.44377591],
       [ 4.37794766,  5.71748756, 76.86280565, ...,  1.49810239,
         1.36333434,  1.44377591],
       ...,
       [ 5.0064649 ,  5.80322308, 77.86704331, ...,  0.        ,
         0.        ,  0.        ],
       [ 5.09315693,  5.7817892 , 77.94429237, ...,  0.        ,
         0.        ,  0.        ],
       [ 5.24486799,  5.77107226, 78.02154142, ...,  0.        ,
         0.        ,  0.        ]])

In [40]:
import joblib
joblib.dump(sc,'cyclone')

['cyclone']

In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [42]:
from sklearn.ensemble import RandomForestRegressor

In [43]:
mr= RandomForestRegressor(n_estimators=1000,criterion="mse",random_state=0)
mr.fit(x_train,y_train)

<ipython-input-43-84cba359e344>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  mr.fit(x_train,y_train)


RandomForestRegressor(n_estimators=1000, random_state=0)

In [44]:
joblib.dump(mr,'cyclone.save')

['cyclone.save']

In [45]:
y_pred = mr.predict(x_test)
y_pred

array([ 7.12005079,  9.47985   , 10.38443651, ...,  3.184     ,
       11.        , 10.75165952])

In [46]:
df

,Status,Latitude,Longitude,Minimum Pressure,Wind NW,Wind NE,Wind SW,Wind SE
0,11,20.2,106.3,995,17.0,19.333333,13.0,16.666667
1,11,20.2,106.4,995,17.0,19.333333,13.0,16.666667
2,11,20.2,106.7,995,17.0,19.333333,13.0,16.666667
3,11,20.3,107.7,995,17.0,19.333333,13.0,16.666667
4,11,20.4,108.6,995,17.0,19.333333,13.0,16.666667
...,...,...,...,...,...,...,...,...
26132,5,21.7,109.0,1002,0.0,20.000000,0.0,0.000000
26133,5,22.4,108.7,1007,0.0,0.000000,0.0,0.000000
26134,5,23.1,108.3,1008,0.0,0.000000,0.0,0.000000
26135,5,23.5,107.9,1009,0.0,0.000000,0.0,0.000000


In [47]:
a=mr.predict([[41.3,50.4,981,40,113.33,80,113.33]])
a

array([7.288])

In [48]:
a=mr.predict([[28.00,94.8,992,27.666667,38.3333,23,35.666667]])
a

array([10.199])

In [49]:
a=mr.predict([[14.4644,75.9218,50,418,3.33,36,3.33]])
a

array([3.872])

In [50]:
a=mr.predict([[14.4644,75.9218,50,418,3.33,36,3.33]])
a

array([3.872])

In [51]:
a=mr.predict([[14.4644,75.9218,1000,180,133.33,346,30.33]])
a

array([10.61])

In [52]:
a=mr.predict([[74.4644,45.9218,1000,180,133.33,346,30.33]])
a

array([2.043])